## Benchmark: sea surface temperature

In [ ]:
# imports
import os
import sys
import torch
import pickle
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import torch_cae_multilevel_V4 as net
import torch_cae_skip_connection as net2

In [ ]:
# load data
data_path = '../data/npy/sst.mnmean.mask2.npy'
map_path = '../data/npy/mask_w1.npy'

dataset = net.MultiScaleDynamicsDataSet(data_path, n_levels=4)

### multilevel CAE with progressive refinements

In [ ]:
archs = [[1,3,5,7],[1,5,9,13],[1,9,17,25],[1,17,33,49]]
tols = [0.05, 0.03, 0.02, 0.01]
net.train_net(archs=archs, dataset=dataset, max_epoch=4000, batch_size=350, tols=tols, activation=torch.nn.Sequential(), 
              w=0.5, model_path=model_path, result_path=result_path, std=0.01, verbose=2)

In [ ]:
# result_path = '../result/sst'
# results_1 = {}
# for file_name in sorted(os.listdir(result_path)):
#     if file_name.endswith('.dat'):
#         key, _ = file_name.split('.')
#         with open(os.path.join(result_path, file_name), 'rb') as f: 
#             records[key]= pickle.load(f)

### multilevel CAE with nonlinear activation functions (without progressive refinements)

In [ ]:
arch = results_1['arch']
results_2 = net.train_net(arch=arch, dataset=dataset, max_epoch=5000, batch_size=315, tols=None,
                          activation=torch.nn.ReLU(),  w=0.9, std=0.01)

### CAE with symmetric skipped connections & nonlinear activations

In [ ]:
arch = results_1['arch']
n_params_3, errs_3 = net2.train_archs(arch=arch, activation=torch.nn.ReLU(), dataset=dataset, 
                                      base_epoch=5000, batch_size=315, w=0.9, std=0.01)

### Linear CAE with symmetric skipped connections

In [ ]:
arch = results_1['arch']
n_params_4, errs_4 = net2.train_archs(arch=arch, activation=torch.nn.Sequential(), dataset=dataset, 
                                      base_epoch=5000, batch_size=315, w=0.9, std=0.01)

### err - # of params 

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(np.log(results_1['n_params']), np.log(results_1['best_val_errs']), 'r-o', markersize=10, linewidth=2.0)
plt.plot(np.log(results_2['n_params']), np.log(results_2['best_val_errs']), 'b-o', markersize=10, linewidth=2.0)
plt.plot(np.log(n_params_3), np.log(errs_3), 'g-o', markersize=10, linewidth=2.0)
plt.plot(np.log(n_params_4), np.log(errs_4), 'c-o', markersize=10, linewidth=2.0)
plt.xlabel('log(number of parameters)', fontsize=20)
plt.ylabel('log(validation error)', fontsize=20)

### err - # of encodings

In [ ]:
# compute the size of encodings
arch_diff = list()
for l in arch:
    arch_diff.append([1] + [l[i] - l[i-1] for i in range(1, len(l))])

size_of_maps = list()
for i in range(len(arch)):
    size_of_maps.append([np.multiply(*results_1['model'].resolved_maps[str(i)]['0'].size())])
    for j in range(len(results_1['model'].resolved_maps[str(i)]) - 1):
        n1 = int(torch.sum(1 - results_1['model'].resolved_maps[str(i)][str(j)]))
        size_of_maps[i].append(n1)

n_encodings_1 = [0]
n_encodings_234 = [0]
for i in range(len(arch_diff)):
    n_encodings_1.append(n_encodings_1[-1] + size_of_maps[i][0])
    n_encodings_234.append(n_encodings_234[-1] + size_of_maps[i][0])
    for j in range(1, len(arch_diff[i])):
        # add_size = min(size_of_maps[i][j]*(2+arch_diff[i][j]), size_of_maps[i][0]*arch_diff[i][j])
        add_size = size_of_maps[i][j] * arch_diff[i][j]
        n_encodings_1.append(n_encodings_1[-1] + add_size)
        n_encodings_234.append(n_encodings_234[-1] + size_of_maps[i][0]*arch_diff[i][j])

n_encodings_1 = n_encodings_1[1:]
n_encodings_234 = n_encodings_234[1:]

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(np.log(n_encodings_1), np.log(results_1['best_val_errs']), 'r-o', markersize=10, linewidth=2.0)
plt.plot(np.log(n_encodings_234), np.log(results_2['best_val_errs']), 'b-o', markersize=10, linewidth=2.0)
plt.plot(np.log(n_encodings_234), np.log(errs_3), 'k-o', markersize=10, linewidth=2.0)
plt.plot(np.log(n_encodings_234), np.log(errs_4), 'c-o', markersize=10, linewidth=2.0)
plt.xlabel('log(size of encodings)', fontsize=20)
plt.ylabel('log(validation error)', fontsize=20)